In [6]:
# Standard imports. Note: You must pip install nasdaqdatalink 1st
import os
import pandas as pd
import nasdaqdatalink
import requests
import shutil
import hvplot.pandas

from pathlib import Path
from MCForecastTools import MCSimulation

In [7]:
# Linking my API key to .env in the same folder. The key is stored in the folder without any quotations around it 

nasdaqdatalink.read_key(filename=".env")

In [8]:
# A function to retrieve a dataframe of counties, zips, etc
def get_regions(regions):
    region_df=nasdaqdatalink.get_table('ZILLOW/REGIONS', region_type=regions)  
    return region_df

# 1. Get the regions data from Zillow REST APIs.   
This contains a list of all counties in the US.

In [9]:
# Using get_regions to retrieve a list of counties
region_df = get_regions('county')
region_df[["county", "state"]] = region_df["region"].str.split(';', 1, expand=True)
region_df["state"] = region_df["state"].str.split(';', 1, expand=True)[0]

#
# Clean up regions data
# Remove ' County' so that we can match the Zillow data with Wikipedia data.
region_df["county"] = region_df["county"].str.replace(" County", "")

# Remove the leading blank space from the 'state' column.
region_df["state"] = region_df['state'].str[1:]

# Clean up region_id datatype.
region_df['region_id']=region_df['region_id'].astype(int)

# Check data for region_df
display(region_df.head())
display(region_df.tail())

,region_id,region_type,region,county,state
None,,,,,
0,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC
1,998,county,Duplin County; NC,Duplin,NC
2,997,county,Dubois County; IN; Jasper,Dubois,IN
3,995,county,Donley County; TX,Donley,TX
4,993,county,Dimmit County; TX,Dimmit,TX


,region_id,region_type,region,county,state
None,,,,,
2886,1003,county,Elmore County; AL; Montgomery,Elmore,AL
2887,1002,county,Elbert County; GA,Elbert,GA
2888,1001,county,Elbert County; CO; Denver-Aurora-Lakewood,Elbert,CO
2889,1000,county,Echols County; GA; Valdosta,Echols,GA
2890,100,county,Bibb County; AL; Birmingham-Hoover,Bibb,AL


# 2. Get the Zillow sales data.  
In this example, we read in Zillow sales data in the form of a CSV file.  

In [10]:
# Get the Zillow sales data. 
# The actual API call using the SDK.
# Instructions can be found here https://data.nasdaq.com/databases/ZILLOW/usage/quickstart/python
# Replace 'quandl' w/ 'nasdaqdatalink
# Turned into a function to prevent constant re-downloading massive csv

def get_zillow_data():
    data = nasdaqdatalink.export_table('ZILLOW/DATA', indicator_id='ZSFH', region_id=list(region_df['region_id']),filename='db.zip')
    
    # Unzipping database from API call
    shutil.unpack_archive('db.zip')
    return data        

In [11]:
# Reading in Database
zillow_data=pd.read_csv(
    Path('ZILLOW_DATA_d5d2ff90eb7172dbde848ea36de12dfe.csv')
)

# Check the Zillow sales data
display(zillow_data.head())
display(zillow_data.tail())

,indicator_id,region_id,date,value
0,ZSFH,100,2007-11-30,123760.0
1,ZSFH,100,2007-12-31,123754.0
2,ZSFH,100,2008-01-31,123605.0
3,ZSFH,100,2008-02-29,123393.0
4,ZSFH,100,2008-03-31,123095.0


,indicator_id,region_id,date,value
669311,ZSFH,999,2022-02-28,390111.0
669312,ZSFH,999,2022-03-31,401621.0
669313,ZSFH,999,2022-04-30,411421.0
669314,ZSFH,999,2022-05-31,422028.0
669315,ZSFH,999,2022-06-30,430509.0


In [12]:
## Merge the Region dataframe with the Zillow sales data
zillow_merge_df = pd.merge(region_df, zillow_data, on=['region_id'])

# Check the merged Zillow data
zillow_merge_df.head()

,region_id,region_type,region,county,state,indicator_id,date,value
0,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-02-28,139430.0
1,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-03-31,139459.0
2,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-04-30,139659.0
3,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-05-31,139887.0
4,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-06-30,140303.0


# 3. Get the county coordinates data.
We couldn't find the county coordinates from Zillow, so we sourced the data from Wikipedia.   We are going to have to merge the data with Zillow based on county and state. 

In [13]:
# Read in county data with coordinates
county_coordinates_df=pd.read_csv(
    Path('counties_w_coordinates.csv')
)

# Clean up data.
# We need to rename the columns so that we can merge our Zillow data set 
# with the county coordinates data.   The dataframes will be merged against 'county' and 'state'. 
county_coordinates_df = county_coordinates_df.rename(columns={"County\xa0[2]" : "county"})
# county_coordinates_df = county_coordinates_df.rename(columns={"region" : "region"})
county_coordinates_df = county_coordinates_df.rename(columns={"State" : "state"})

# Remove degrees 
county_coordinates_df["Latitude"] = county_coordinates_df["Latitude"].str.replace("°", "")
county_coordinates_df["Longitude"] = county_coordinates_df["Longitude"].str.replace("°", "")

# Remove + sign for Latitude and Longitude
county_coordinates_df["Latitude"] = county_coordinates_df["Latitude"].str.replace("+", "")
county_coordinates_df["Longitude"] = county_coordinates_df["Longitude"].str.replace("+", "")

# Some of the data uses unicode hyphens which causes problems when trying to convert the Longitude and Latitude to float.
county_coordinates_df["Latitude"] = county_coordinates_df["Latitude"].str.replace('\U00002013', '-')
county_coordinates_df["Longitude"] = county_coordinates_df["Longitude"].str.replace('\U00002013', '-')

# Convert Longitude and Latitude to float so we can display on the map. 
county_coordinates_df["Latitude"] = county_coordinates_df["Latitude"].astype(float)
county_coordinates_df["Longitude"] = county_coordinates_df["Longitude"].astype(float)

# Check the county coordinates data
county_coordinates_df.head()

C:\Users\potte\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
C:\Users\potte\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


,Sort [1],state,FIPS,county,County Seat(s) [3],Population,Land Area,Land Area.1,Water Area,Water Area.1,Total Area,Total Area.1,Latitude,Longitude
0,NaN,NaN,NaN,NaN,NaN,-2010,km²,mi²,km²,mi²,km²,mi²,NaN,NaN
1,1.0,AL,1001.0,Autauga,Prattville,"54,571","1,539.58",594.436,25.776,9.952,"1,565.36",604.388,32.536382,-86.644490
2,2.0,AL,1003.0,Baldwin,Bay Minette,"182,265","4,117.52","1,589.78","1,133.19",437.527,"5,250.71","2,027.31",30.659218,-87.746067
3,3.0,AL,1005.0,Barbour,Clayton,"27,457","2,291.82",884.876,50.865,19.639,"2,342.68",904.515,31.870670,-85.405456
4,4.0,AL,1007.0,Bibb,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,33.015893,-87.127148


In [14]:
# Merge the Zillow data and county coordinates data.
master_df = pd.merge(zillow_merge_df, county_coordinates_df, on=['county', 'state'])

# Check the master data
master_df

,region_id,region_type,region,county,state,indicator_id,date,value,Sort [1],FIPS,County Seat(s) [3],Population,Land Area,Land Area.1,Water Area,Water Area.1,Total Area,Total Area.1,Latitude,Longitude
0,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-02-28,139430.0,1922.0,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,36.036589,-78.877919
1,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-03-31,139459.0,1922.0,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,36.036589,-78.877919
2,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-04-30,139659.0,1922.0,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,36.036589,-78.877919
3,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-05-31,139887.0,1922.0,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,36.036589,-78.877919
4,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-06-30,140303.0,1922.0,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,36.036589,-78.877919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628666,100,county,Bibb County; AL; Birmingham-Hoover,Bibb,AL,ZSFH,2022-02-28,161462.0,4.0,1007.0,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,33.015893,-87.127148
628667,100,county,Bibb County; AL; Birmingham-Hoover,Bibb,AL,ZSFH,2022-03-31,162369.0,4.0,1007.0,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,33.015893,-87.127148
628668,100,county,Bibb County; AL; Birmingham-Hoover,Bibb,AL,ZSFH,2022-04-30,163859.0,4.0,1007.0,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,33.015893,-87.127148
628669,100,county,Bibb County; AL; Birmingham-Hoover,Bibb,AL,ZSFH,2022-05-31,164684.0,4.0,1007.0,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,33.015893,-87.127148


# 4. Display in a Map

In [1]:
# Get mean data by state and county
county_df = master_df.groupby(["state", "county"]).mean()

# Divide price by 1000 so that it looks better on map.
county_df["value"] = county_df["value"] /10000

county_df['value'] = county_df['value'].pct_change()
county_df = county_df.dropna()

# Check data
display(county_df.head())
display(county_df.tail())

NameError: name 'master_df' is not defined

In [16]:
county_df.hvplot.points(
    'Longitude',
    'Latitude',
    geo=True,
    size='value',
    color='value',
    tiles='OSM',
    height=700,
    width=1200)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (value)

# The MAC/D

In [17]:
filtered_df = master_df[['date','county','state','value']]
filtered_df = filtered_df.sort_values('value', ascending=False)

In [18]:
# Monte Carlo Simulation

In [19]:
# Deleting county and state columns and replacing with location column which contains county, state
# This is necessary because same county names exist in different states. 
mc_df = filtered_df
mc_df['location'] = mc_df['county'] + ", " + mc_df['state']
drop_cols = ['county', 'state']
mc_df = mc_df.drop(columns=drop_cols)
mc_df.set_index(mc_df['location'])
mc_df.shape

(628671, 3)

In [20]:
# Getting average home value for each location
values_df = mc_df.groupby('location', as_index=False)['value'].mean()
values_df = values_df.sort_values(by='value')
list_of_all_counties = values_df['location'].tolist()
list_of_all_counties.sort()
highest_df = values_df.tail(10)
display(highest_df)
#display(list_of_all_counties)

,location,value
1706,"Monroe, FL",6.540228e+05
2354,"Summit, UT",6.736183e+05
2193,"San Miguel, CO",7.668141e+05
2204,"Santa Clara, CA",7.722854e+05
2403,"Teton, WY",8.507195e+05
2192,"San Mateo, CA",8.627426e+05
1555,"Marin, CA",8.900239e+05
2184,"San Francisco, CA",9.046193e+05
1782,"Nantucket, MA",1.220883e+06
1988,"Pitkin, CO",2.156456e+06


In [21]:
location_group_df = mc_df.copy()
location_group_df.set_index(['location', 'date'], inplace=True)
display(location_group_df)
display(mc_df)

value
location   date                 
Pitkin, CO 2022-06-30  3976156.0
           2022-05-31  3896466.0
           2022-04-30  3871460.0
           2022-02-28  3826409.0
           2022-03-31  3819310.0
...                          ...
Pike, KY   1999-03-31    19160.0
           1998-11-30    19154.0
           1999-01-31    19147.0
           1998-06-30    19140.0
           1998-10-31    19133.0

[628671 rows x 1 columns]

,date,value,location
557880,2022-06-30,3976156.0,"Pitkin, CO"
557879,2022-05-31,3896466.0,"Pitkin, CO"
557878,2022-04-30,3871460.0,"Pitkin, CO"
557876,2022-02-28,3826409.0,"Pitkin, CO"
557877,2022-03-31,3819310.0,"Pitkin, CO"
...,...,...,...
239361,1999-03-31,19160.0,"Pike, KY"
239357,1998-11-30,19154.0,"Pike, KY"
239359,1999-01-31,19147.0,"Pike, KY"
239352,1998-06-30,19140.0,"Pike, KY"


In [22]:
most_expensive_counties = highest_df['location'].to_numpy()
most_expensive_counties

array(['Monroe, FL', 'Summit, UT', 'San Miguel, CO', 'Santa Clara, CA',
       'Teton, WY', 'San Mateo, CA', 'Marin, CA', 'San Francisco, CA',
       'Nantucket, MA', 'Pitkin, CO'], dtype=object)

In [23]:
lowest_df = values_df.head(10)
least_expensive_counties = lowest_df['location'].to_numpy()
least_expensive_counties

array(['Phillips, AR', 'Tillman, OK', 'Jefferson, OK', 'Allendale, SC',
       'Pike, KY', 'Leflore, MS', 'Wyoming, WV', 'Lee, AR',
       'Woodruff, AR', 'Clay, KY'], dtype=object)

In [18]:
mc_df = mc_df.sort_values(by='date')
dataframe_array = []
start_date = '2015-01-31'
end_date = '2022-06-30'
for group_loc in most_expensive_counties:
    df_temp = mc_df.loc[(mc_df['location']==group_loc) & (mc_df['date'] <= end_date) & (mc_df['date'] >= start_date)]
    dataframe_array.append(df_temp.drop('location', axis=1).reset_index())
    
#dataframe_array
newest_df = pd.concat(dataframe_array, axis=1, keys=['Monroe, FL', 'Summit, UT', 'San Miguel, CO', 'Santa Clara, CA',
       'Teton, WY', 'San Mateo, CA', 'Marin, CA', 'San Francisco, CA',
       'Nantucket, MA', 'Pitkin, CO'])
newest_df.tail(90)

KeyError: 'location'

In [ ]:
mc_data = MCSimulation(newest_df)

In [ ]:
display(mc_data.calc_cumulative_return())

In [ ]:
mc_data.plot_simulation()

In [ ]:
mc_data.plot_distribution()

In [ ]:
mc_data.summarize_cumulative_return()